In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [ ]:
# loop into data folder
data_folder = '../data'
file_paths = {}
for file in os.listdir(data_folder):
    if file.endswith('.csv'):
        location = file.split('.')[0]
        file_paths[location] = os.path.join(data_folder, file)

In [ ]:
# Load datasets into a dictionary
datasets = {}
for sensor, path in file_paths.items():
    print(sensor)
    datasets[sensor] = pd.read_csv(path, parse_dates=['valid_at'], index_col='valid_at')

    if 'sensor_id' not in datasets[sensor].columns:
        datasets[sensor]['sensor_id'] = 'one_sensor'

In [ ]:
num_datasets = len(datasets)
num_features = len(next(iter(datasets.values())).columns)

fig, axes = plt.subplots(num_features, num_datasets, figsize=(4 * num_datasets, 4 * num_features), sharey=False)

# Loop over each dataset and feature to create subplots
for col_idx, feature in enumerate(next(iter(datasets.values())).columns):
    print(feature)
    for row_idx, (location, df) in enumerate(datasets.items()):
        if feature not in df.columns:
            continue
        sns.histplot(df[feature], kde=True, ax=axes[col_idx, row_idx])  # Plotting histogram with KDE
        axes[col_idx, row_idx].set_title(f'{location} - {feature}')
        axes[col_idx, row_idx].set_xlabel('')  # Clear x-axis labels for better readability

# Layout adjustment
plt.tight_layout()
plt.show()

In [ ]:
data_dict = datasets.copy()


In [ ]:
data_concat = pd.concat({k: v.reset_index(drop=True) for k, v in data_dict.items()}, names=["Location"])

In [ ]:
data_concat

In [ ]:
data_dict

In [ ]:
# Initialize a list to store the correlations
correlation_data = []

# drop sensor_id
try:
    data_concat = data_concat.drop('sensor_id', axis=1)
except KeyError:
    pass

try:
    data_dict = {k: v.drop('sensor_id', axis=1) for k, v in data_dict.items()}
except KeyError:
    pass

# Calculate correlations for each dataset
for location, df in data_dict.items():
    print(location, '----------------')
    print(df.head())
    # Compute pairwise correlations numeric columns
    numeric_df = df.select_dtypes(include='number')  # Select numeric columns only
    corr_matrix = numeric_df.corr()     
    
    corr_pairs = corr_matrix.unstack().reset_index()  # Flatten the matrix
    corr_pairs.columns = ['Feature1', 'Feature2', 'Correlation']
    corr_pairs = corr_pairs[corr_pairs['Feature1'] != corr_pairs['Feature2']]  # Exclude self-correlations
    corr_pairs['Location'] = location  # Add location for identification
    correlation_data.append(corr_pairs)

# Concatenate all correlation data into a single DataFrame
correlation_df = pd.concat(correlation_data, ignore_index=True)

# Plot the boxplot for each location's feature correlations
plt.figure(figsize=(12, 6))
sns.boxplot(data=correlation_df, x='Location', y='Correlation')
plt.title('Distribution of Feature Correlations by Location')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize dictionary to store correlation lists for each feature
feature_correlation_data = {}

# Calculate correlations for each feature in each dataset
for location, df in data_dict.items():
    numeric_df = df.select_dtypes(include='number')  # Select numeric columns only
    corr_matrix = numeric_df.corr()  # Compute pairwise correlations

    for feature in numeric_df.columns:
        # Gather correlations for each feature (excluding self-correlation)
        feature_corrs = corr_matrix[feature].drop(feature).dropna()
        
        if feature not in feature_correlation_data:
            feature_correlation_data[feature] = []
            
        # Store correlations and associated location
        for corr in feature_corrs:
            feature_correlation_data[feature].append({'Location': location, 'Correlation': corr})

# Convert to DataFrame
correlation_df = pd.DataFrame([
    {'Feature': feature, 'Location': item['Location'], 'Correlation': item['Correlation']}
    for feature, correlations in feature_correlation_data.items()
    for item in correlations
])

# Plot the boxplot for each feature's correlations across datasets
plt.figure(figsize=(12, 6))
sns.boxplot(data=correlation_df, x='Feature', y='Correlation', hue='Location')
plt.title('Distribution of Feature Correlations by Location')
plt.xticks(rotation=45)
plt.legend(title='Location')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Dictionary to store correlations for each feature pair across locations
pairwise_correlations = {}

# Calculate correlations for each pair of features in each dataset
for location, df in data_dict.items():
    numeric_df = df.select_dtypes(include='number')
    corr_matrix = numeric_df.corr()
    
    for i, feature1 in enumerate(numeric_df.columns):
        for feature2 in numeric_df.columns[i+1:]:  # Avoid duplicate pairs and self-correlation
            pair = f'{feature1}-{feature2}'
            correlation_value = corr_matrix.loc[feature1, feature2]
            
            if pair not in pairwise_correlations:
                pairwise_correlations[pair] = []
            
            pairwise_correlations[pair].append({'Location': location, 'Correlation': correlation_value})

# Convert to a DataFrame for plotting
correlation_df = pd.DataFrame([
    {'Feature Pair': pair, 'Location': item['Location'], 'Correlation': item['Correlation']}
    for pair, correlations in pairwise_correlations.items()
    for item in correlations
])

# Plot the boxplot for each feature pair's correlations across locations
plt.figure(figsize=(14, 8))
sns.boxplot(data=correlation_df, x='Feature Pair', y='Correlation')
plt.xticks(rotation=90)
plt.title('Distribution of Feature Pair Correlations Across Locations')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Specify the target features
target_features = ['pm2p5_x', 'pm2p5_y', 'temperature', 'relative_humidity', 'pressure']
pairwise_correlations = {}

# Calculate correlations for each specified feature pair in each dataset
for location, df in data_dict.items():
    numeric_df = df[target_features].select_dtypes(include='number')  # Filter only target features
    corr_matrix = numeric_df.corr()
    
    for i, feature1 in enumerate(target_features):
        for feature2 in target_features[i+1:]:  # Avoid duplicate pairs
            pair = f'{feature1}-{feature2}'
            correlation_value = corr_matrix.loc[feature1, feature2]
            
            if pair not in pairwise_correlations:
                pairwise_correlations[pair] = []
            
            pairwise_correlations[pair].append({'Location': location, 'Correlation': correlation_value})

# Convert to a DataFrame for plotting
correlation_df = pd.DataFrame([
    {'Feature Pair': pair, 'Location': item['Location'], 'Correlation': item['Correlation']}
    for pair, correlations in pairwise_correlations.items()
    for item in correlations
])

# Plot the boxplot for each feature pair's correlations across locations
plt.figure(figsize=(12, 6))
sns.boxplot(data=correlation_df, x='Feature Pair', y='Correlation')
plt.xticks(rotation=45)
plt.title('Distribution of Specified Feature Pair Correlations Across Locations')
plt.tight_layout()
plt.show()


In [ ]:
feature_correlation_data 

In [ ]:
# Plot
# num_features = len(data_concat.columns) - 2  # exclude location
# fig, axes = plt.subplots(num_features, 1, figsize=(10, 4 * num_features))

# for idx, feature in enumerate(data_concat.columns[2:]):  # skip 'Location'
#     print(feature)
sns.violinplot(data=data_concat, x="Location", y='pm2p5_x')
# plt.set_title(f'Distribution of pm2p5_x by Location')
# plt.set_xlabel('')

plt.tight_layout()
plt.show()


In [ ]:
# Plot
num_features = len(data_concat.columns) - 2  # exclude location
fig, axes = plt.subplots(4, 1, figsize=(10, 4 * 4))

for idx, feature in enumerate(['temperature', 'relative_humidity', 'pm2p5_x', 'pm2p5_y']):  # skip 'Location'
    print(feature)
    sns.violinplot(data=data_concat, x="Location", y=feature, ax=axes[idx])
    axes[idx].set_title(f'Distribution of {feature} by Location')
    axes[idx].set_xlabel('')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Plot
num_features = len(data_concat.columns) - 2  # exclude location
fig, axes = plt.subplots(4, 1, figsize=(10, 4 * 4))

for idx, feature in enumerate(['temperature', 'relative_humidity', 'pm2p5_x', 'pm2p5_y']):  # skip 'Location'
    sns.boxenplot(data=data_concat, x="Location", y=feature, ax=axes[idx])
    axes[idx].set_title(f'Distribution of {feature} by Location')
    axes[idx].set_xlabel('')

plt.tight_layout()
plt.show()


In [ ]:
data_concat

In [ ]:
data_concat['Location'] = data_concat.index.get_level_values(0)
for feature in ['temperature', 'relative_humidity', 'pm2p5_x', 'pm2p5_y']:  # each feature
    g = sns.FacetGrid(data_concat, col="Location", height=4, aspect=1)
    g.map(sns.histplot, feature, kde=True)
    g.fig.suptitle(f'Distribution of {feature} by Location', y=1.05)
    plt.show()


In [ ]:
datasets['aosta']

In [ ]:
summary_data = []

# Now you can loop through the datasets
for location, df in datasets.items():
    for sensor_id, df_sensor in df.groupby('sensor_id'):
        # Calculate the min, max, number of data points, and reading frequency
        min_value = df_sensor['valid_at'].min()
        max_value = df_sensor['valid_at'].max()
        num_data_points = len(df_sensor)

        # Assuming the dataset has a 'timestamp' column, calculate frequency of readings
        # Convert timestamp to datetime if not already
        df['valid_at'] = pd.to_datetime(df_sensor['valid_at'])

        # Calculate the min and max difference between consecutive readings 
        min_diff = df_sensor['valid_at'].diff().min()
        max_diff = df_sensor['valid_at'].diff().max()

        # Append to the summary list
        summary_data.append({
            'Location': str.capitalize(location) + ' - ' + str(sensor_id),
            'Min Datetime': min_value,
            'Max Datetime': max_value,
            'Num Data Points': num_data_points,
            'Num sensors': len(df['sensor_id'].unique()),
            # 'Frequency (Avg Datetime Diff)': str(min_diff) + ' - ' + str(max_diff)
        })

# Convert summary data to DataFrame
summary_df = pd.DataFrame(summary_data)

# Sort the DataFrame by Min Value
summary_df = summary_df.sort_values(by='Min Datetime')

# Display the summary table
print(summary_df)

# Plot the table as an image
fig, ax = plt.subplots(figsize=(10, 3))  # Adjust the size to fit the table
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=summary_df.values, colLabels=summary_df.columns, loc='center')

# Set font size for the entire table
table.auto_set_font_size(False)  # Disable automatic font size
table.set_fontsize(8)  # Set font size (adjust as needed)

# Adjust cell width for better readability
# table.scale(1.5, 1.5)  # Scale the table (1.5x width and height)

plt.show()

In [ ]:
columns = ['pm2p5_x', 'pressure', 'relative_humidity', 'temperature', 'wind_speed', 'rain', 'pm2p5_y']

In [ ]:
# Loop through each column and plot the distribution (ignoring non-numeric columns)
for column in columns:
    plt.figure(figsize=(10, 6))  # Create a new figure for each dataset
    plt.title(f'Distribution of column {column}')
    for location, df in datasets.items():
        # Plot the distribution (using hist or density plot)
        df[column].plot(kind='density', label=location, alpha=0.7)

    # Show legend and labels
    plt.legend()
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.show()

In [ ]:
# for each location plot the difference between pm2p5_x and pm2p5_y based on rh ranges
# Define RH bins and labels
bins = [0, 60, 70, 80, 90, 100]
labels = ['0-60', '60-70', '70-80', '80-90', '90-100']

# Loop through each dataset

for location, df in datasets.items():
    plt.figure(figsize=(10, 6))  # Create figure for each dataset

    # Create a new column for RH categories based on the bins
    df['rh_category'] = pd.cut(df['relative_humidity'], bins=bins, labels=labels, include_lowest=True)

    # Plot boxplot for pm2p5_x grouped by rh_category
    df.boxplot(column='pm2p5_x', by='rh_category', grid=False, vert=False, showfliers=False)

    plt.title(f'Boxplot of pm2p5_x grouped by RH ranges for {location}')
    plt.suptitle('')  # Remove default title to avoid overlap
    plt.xlabel('PM 2.5 (µg/m³)')
    plt.ylabel('RH Range')
    plt.show()

In [ ]:
datasets

In [ ]:
# for each location plot the difference between pm2p5_x and pm2p5_y based on rh ranges in a single plot
# Define RH bins and labels
bins = [0, 60, 70, 80, 90, 100]
labels = ['0-60', '60-70', '70-80', '80-90', '90-100']

# Combine datasets into a single DataFrame
combined_data = pd.concat(
    [df.assign(location=location) for location, df in datasets.items()],
    ignore_index=True  # Reset index
)

# Create a new column for RH categories based on the bins
combined_data['rh_category'] = pd.cut(combined_data['relative_humidity'], bins=bins, labels=labels, include_lowest=True)

# Calculate the difference between pm2p5_x and pm2p5_y
combined_data['pm2p5_diff'] = combined_data['pm2p5_x'] - combined_data['pm2p5_y']

# Set up the plot
plt.figure(figsize=(12, 8))

# Create boxplot with RH categories on x-axis, PM difference on y-axis, and colored by location
sns.boxplot(data=combined_data, x='rh_category', y='pm2p5_diff', hue='location', palette='Set2', showfliers=False)

plt.title('Boxplot of PM2.5 Difference (PM 2.5 LCS - PM 2.5 LS) by RH Ranges')
plt.xlabel('RH Range')
plt.ylabel('PM 2.5 Difference (µg/m³)')
plt.axhline(0, color='grey', linestyle='--')  # Optional: line at zero for reference
plt.legend(title='Location', bbox_to_anchor=(1.05, 1), loc='upper left')  # Adjust legend position
plt.show()

In [ ]:
# for each location scatter pm2p5_x and pm2p5_y based on rh scale
# Define RH bins and labels
# Loop through each dataset
for location, df in datasets.items():
    plt.figure(figsize=(10, 6))  # Create figure for each dataset

    # Create a new column for RH categories based on the bins
    # df['rh_category'] = pd.cut(df['relative_humidity'], bins=bins, labels=labels, include_lowest=True)

    # Plot scatter plot for pm2p5_x and pm2p5_y colored by relative humidity
    plt.scatter(df['pm2p5_x'], df['pm2p5_y'], c=df['relative_humidity'], cmap='coolwarm', alpha=0.7)

    # color map from 0 to 100
    plt.colorbar(label='Relative Humidity (%)')

    # fix color map scale from 0 to 100
    plt.clim(0, 100)

    plt.title(f'Scatter Plot of pm2p5_x vs pm2p5_y colored by RH for {location}')
    plt.xlabel('PM 2.5 Low-cost (µg/m³)')
    plt.ylabel('PM 2.5 Reference (µg/m³)')
    plt.show()

In [ ]:
# Loop through each column
for column in columns:
    plt.figure(figsize=(10, 6))  # Create a new figure for each column
    plt.title(f'Box Plot of column {column}')

    # Collect data from all locations for this column
    data = [df[column].dropna() for location, df in datasets.items()]  # Drop NaNs for boxplot

    # Create boxplot for the column across different datasets
    plt.boxplot(data, tick_labels=datasets.keys(), showfliers=False)  # Hide outliers

    # Show labels
    plt.ylabel('Value')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.show()

In [ ]:
pms = {
    'pm2p5_x': 'Low-cost PM 2.5',
    'pm2p5_y': 'Reference PM 2.5'
}

pms_21102024 = {
    'aosta': 8,
    'badajoz': 6.7,
    'bangalore': 31.2,
    'calgary': 0,
    'delhi': 175,
    'hamirpur': 46,
    'torino': 6,
    'lima_airbeam': 10,
    'lima_iqair': 10,
    'uk_pms5003': 6.7,
    'uk_sps030': 6.7
}

# Loop through each column
for column in pms:
    plt.figure(figsize=(10, 6))  # Create a new figure for each column
    column_name = pms[column]

    plt.title(f'Box Plot of {column_name}')

    # Collect data from all locations for this column
    data = [df[column].dropna() for location, df in datasets.items()]  # Drop NaNs for boxplot

    # Create boxplot for the column across different datasets
    plt.boxplot(data, tick_labels=datasets.keys(), showfliers=False)  # Hide outliers

    # color background based on AQI levels and write the labels

    # Good (0-50): Green
    plt.axhspan(0, 50, color='green', alpha=0.2)
    plt.text(1, 45, 'Good', fontsize=10, verticalalignment='top')
    # Moderate (51-100): Yellow
    plt.axhspan(50, 100, color='yellow', alpha=0.2)
    plt.text(1, 95, 'Moderate', fontsize=10, verticalalignment='top')
    # Unhealthy for Sensitive Groups (101-150): Orange
    plt.axhspan(100, 150, color='orange', alpha=0.2)
    plt.text(1, 145, 'Unhealthy for Sensitive Groups', fontsize=10, verticalalignment='top')
    # Unhealthy (151-200): Red
    plt.axhspan(150, 200, color='red', alpha=0.2)
    plt.text(1, 195, 'Unhealthy', fontsize=10, verticalalignment='top')
    # Very Unhealthy (201-300): Purple
    plt.axhspan(200, 300, color='purple', alpha=0.2)
    plt.text(1, 295, 'Very Unhealthy', fontsize=10, verticalalignment='top')
    # Hazardous (301-500): Maroon
    plt.axhspan(300, 500, color='maroon', alpha=0.2)
    plt.text(1, 495, 'Hazardous', fontsize=10, verticalalignment='top')

    # Add text labels for AQI levels

    # plt.text(len(datasets.keys()) + 1, 25, 'Good (0-50)', color='green')
    # plt.text(len(datasets.keys()) + 1, 75, 'Moderate (51-100)', color='yellow')
    # plt.text(len(datasets.keys()) + 1, 125, 'Unhealthy for Sensitive Groups (101-150)', color='orange')
    # plt.text(len(datasets.keys()) + 1, 175, 'Unhealthy (151-200)', color='red')
    # plt.text(len(datasets.keys()) + 1, 225, 'Very Unhealthy (201-300)', color='purple')
    # plt.text(len(datasets.keys()) + 1, 275, 'Hazardous (301-500)', color='maroon')

    # plot a dot from pms_21102024 for each location
    for i, location in enumerate(datasets.keys()):
        plt.plot(i + 1, pms_21102024[location], 'ro')

    # Show labels
    plt.ylabel('Value')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.show()

In [ ]:
# Loop through each dataset
for location, df in datasets.items():
    for sensor_id in df['sensor_id'].unique():
        fig, ax1 = plt.subplots(figsize=(10, 6))  # Create figure and first axis

        # Plot pm2p5_y and pm2p5_x on the first y-axis (ax1)
        ax1.set_title(f'pm2p5_y, pm2p5_x, and RH for {location} - Sensor {sensor_id}')
        ax1.plot(df['valid_at'], df['pm2p5_y'], label='Reference station', color='black')
        ax1.plot(df['valid_at'], df['pm2p5_x'], label='Low-cost', color='g')

        ax1.set_xlabel('Time')
        ax1.set_ylabel('PM 2.5 (µg/m³)')
        ax1.tick_params(axis='y')
        ax1.legend(loc='upper left')

        # Create a second y-axis (ax2) to plot rh (relative humidity)
        ax2 = ax1.twinx()
        ax2.plot(df['valid_at'], df['relative_humidity'], label='RH', color='r')

        ax2.set_ylabel('Relative Humidity (%)')
        ax2.tick_params(axis='y', labelcolor='r')
        ax2.legend(loc='upper right')

        # fix from 0 to 100
        ax2.set_ylim(0, 100)

        # plot 70% line
        ax2.axhline(y=70, color='red', linestyle='--', label='70% RH')

        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45)

        # Show the plot
        plt.tight_layout()
        plt.show()

In [ ]:
datasets['aosta']

In [ ]:
plt.ion()
from datetime import datetime

# Loop through each dataset
for location, df in datasets.items():
    for sensor_id, df_sensor in df.groupby('sensor_id'):
        fig, ax1 = plt.subplots(figsize=(10, 6))  # Create figure and first axis

        # Plot pm2p5_y and pm2p5_x on the first y-axis (ax1)
        ax1.set_title(f'pm2p5_y, pm2p5_x, and RH for {location} - Sensor {sensor_id}', fontsize=15)
        ax1.plot(df_sensor.index, df_sensor['pm2p5_y'], label='Reference station', color='black')
        ax1.plot(df_sensor.index, df_sensor['pm2p5_x'], label='Low-cost', color='g')

        ax1.set_xlabel('Time', fontsize=15)
        ax1.set_ylabel('PM 2.5 (µg/m³)', fontsize=15)
        ax1.tick_params(axis='y')
        ax1.legend(loc='upper left')

        # Create a second y-axis (ax2) to plot rh (relative humidity)
        ax2 = ax1.twinx()
        ax2.plot(df_sensor.index, df_sensor['relative_humidity'], label='RH', color='r')

        ax2.set_ylabel('Relative Humidity (%)', fontsize=15)
        ax2.tick_params(axis='y', labelcolor='r')
        ax2.legend(loc='upper right')

        # fix from 0 to 100
        ax2.set_ylim(0, 100)

        ax1.set_ylim(0, 250)

        # plot 70% line
        ax2.axhline(y=70, color='red', linestyle='--', label='70% RH')

        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45)

        if location == 'aosta':
            # create datetime
            min_date = datetime(2024, 2, 4, 0, 0, 0)
            max_date = datetime(2024, 2, 24, 0, 0, 0)
        elif location == 'badajoz':
            min_date = datetime(2021, 3, 12, 0, 0, 0)
            max_date = datetime(2021, 4, 1, 0, 0, 0)
        elif location == 'bangalore':
            min_date = datetime(2019, 6, 21, 0, 0, 0)
            max_date = datetime(2019, 7, 11, 0, 0, 0)
        elif location == 'calgary':
            min_date = datetime(2019, 1, 1, 0, 0, 0)
            max_date = datetime(2019, 1, 21, 0, 0, 0)
        elif location == 'delhi':
            min_date = datetime(2019, 1, 1, 0, 0, 0)
            max_date = datetime(2019, 1, 21, 0, 0, 0)
        elif location == 'hamirpur':
            min_date = datetime(2020, 11, 1, 0, 0, 0)
            max_date = datetime(2020, 11, 21, 0, 0, 0)
        elif location == 'torino':
            min_date = datetime(2022, 2, 7, 0, 0, 0)
            max_date = datetime(2022, 2, 27, 0, 0, 0)
        elif location == 'lima_airbeam':
            min_date = datetime(2021, 12, 1, 0, 0, 0)
            max_date = datetime(2021, 12, 21, 0, 0, 0)
        elif location == 'lima_iqair':
            min_date = datetime(2021, 12, 1, 0, 0, 0)
            max_date = datetime(2021, 12, 21, 0, 0, 0)
        elif location == 'uk_pms5003':
            min_date = datetime(2020, 11, 1, 0, 0, 0)
            max_date = datetime(2020, 11, 21, 0, 0, 0)
        elif location == 'uk_sps030':
            min_date = datetime(2020, 11, 1, 0, 0, 0)
            max_date = datetime(2020, 11, 21, 0, 0, 0)

        # zoom over 2 weeks
        # print(location, df_sensor.index.min(), df_sensor.index.max())
        # min_date = df_sensor.index.max() - pd.Timedelta(days=21)
        ax1.set_xlim(min_date, max_date)

        # increase font size
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.legend(fontsize=15)

        # Show the plot
        plt.tight_layout()
        plt.show()

In [ ]:
# calculate mean gap between pm2p5_x and pm2p5_y under and over RH ranges
# Define RH bins and labels
bins = [0, 50, 60, 70, 80, 100]
labels = ['0-50', '50-60', '60-70', '70-80', '80-100']

# Loop through each dataset
for location, df in datasets.items():
    # Create a new column for RH categories based on the bins
    df['rh_category'] = pd.cut(df['relative_humidity'], bins=bins, labels=labels, include_lowest=True)

    # Calculate the mean gap between pm2p5_x and pm2p5_y for each RH category
    mean_gap = df.groupby('rh_category')['pm2p5_x'].mean() - df.groupby('rh_category')['pm2p5_y'].mean()
    #round to 2 decimal
    mean_gap = mean_gap.round(2)

    # print the mean gap
    print(f'Mean gap between PM 2.5 Low-cost and Reference for {location}')
    print(mean_gap)
    print()
    
    # # plot line plot for each location
    # plt.figure(figsize=(10, 6))
    # plt.plot(mean_gap, marker='o', label=location)
    # 
    # plt.title(f'Mean Gap between PM 2.5 Low-cost and Reference for {location}')
    # plt.xlabel('RH Range')
    # plt.ylabel('Mean Gap (µg/m³)')
    # 
    # # fix y-axis to -10 and 150
    # # plt.ylim(-10, 30)
    # # y axis log scale
    # # 
    # 
    # plt.legend()
    # plt.show()

In [ ]:
# Define RH bins and labels
bins = [0, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100']

# Loop through each dataset
for sensor_name, df in datasets.items():
    plt.figure(figsize=(10, 6))  # Create figure for each dataset

    # Create a new column for RH categories based on the bins
    df['rh_category'] = pd.cut(df['relative_humidity'], bins=bins, labels=labels, include_lowest=True)

    # Plot boxplot for pm2p5_x grouped by rh_category
    df.boxplot(column='pm2p5_x', by='rh_category', grid=False, vert=False, showfliers=False)

    plt.title(f'Boxplot of pm2p5_x grouped by RH ranges for {sensor_name}')
    plt.suptitle('')  # Remove default title to avoid overlap
    plt.xlabel('PM 2.5 (µg/m³)')
    plt.ylabel('RH Range')
    plt.show()

In [ ]:
# Define RH bins and labels
bins = [0, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100']

all_metrics = {}

# Loop through each dataset
for location, df in datasets.items():
    metrics = pd.DataFrame(index=['R2', 'RMSE', 'MSE', 'MAE'])

    for sensor_id, df_sensor in df.groupby('sensor_id'):
        # r2, rmse, mse, mae between pm2p5_x and pm2p5_y
        y_true = df_sensor['pm2p5_y']
        y_pred = df_sensor['pm2p5_x']

        # Calculate metrics
        residuals = y_true - y_pred
        r2 = 1 - (np.sum(residuals ** 2) / np.sum((y_true - y_true.mean()) ** 2))
        rmse = np.sqrt(np.mean(residuals ** 2))
        mse = np.mean(residuals ** 2)
        mae = np.mean(np.abs(residuals))

        # tabelize the metrics
        metrics[location + ' \n ' + sensor_id] = [round(r2, 3), round(rmse, 3), round(mse, 3), round(mae, 3)]

    # Plot the table as an image
    fig, ax = plt.subplots(figsize=(15, 2))  # Adjust the size to fit the table
    # ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=metrics.values, colLabels=metrics.columns, rowLabels=metrics.index, loc='center')

    # Set font size for the entire table
    table.auto_set_font_size(False)  # Disable automatic font size
    table.set_fontsize(16)  # Set font size (adjust as needed)

    # Adjust cell width for better readability
    table.scale(1.5, 1.5)  # Scale the table (1.5x width and height)

    plt.show()

    # Store the metrics
    all_metrics[location] = metrics

In [ ]:
# remove for each sensor_id of each dataset 3 std from the mean
datasets_cleaned = {}

# Loop through each dataset
for location, df in datasets.items():
    for sensor_id, df_sensor in df.groupby('sensor_id'):
        # Calculate the mean and standard deviation of pm2p5_x
        mean_pm2p5_x = df_sensor['pm2p5_x'].mean()
        std_pm2p5_x = df_sensor['pm2p5_x'].std()

        # Remove values outside 3 standard deviations from the mean
        df_sensor['pm2p5_x'] = df_sensor['pm2p5_x'].clip(mean_pm2p5_x - 3 * std_pm2p5_x, mean_pm2p5_x + 3 * std_pm2p5_x)

        # Plot the distribution of pm2p5_x after clipping
        plt.figure(figsize=(10, 6))
        plt.title(f'Distribution of pm2p5_x for {location} - Sensor {sensor_id}')
        sns.histplot(df_sensor['pm2p5_x'], kde=True)
        plt.xlabel('PM 2.5 (µg/m³)')
        plt.ylabel('Density')
        plt.show()

        # Store the cleaned dataset
        if location not in datasets_cleaned:
            datasets_cleaned[location] = pd.DataFrame()
        datasets_cleaned[location] = pd.concat([datasets_cleaned[location], df_sensor])



In [ ]:
all_metrics_cleaned = {}

# Loop through each dataset
for location, df in datasets_cleaned.items():
    metrics_cleaned = pd.DataFrame(index=['R2', 'RMSE', 'MSE', 'MAE'])

    for sensor_id, df_sensor in df.groupby('sensor_id'):
        # r2, rmse, mse, mae between pm2p5_x and pm2p5_y
        y_true = df_sensor['pm2p5_y']
        y_pred = df_sensor['pm2p5_x']

        # Calculate metrics
        residuals = y_true - y_pred
        r2 = 1 - (np.sum(residuals ** 2) / np.sum((y_true - y_true.mean()) ** 2))
        rmse = np.sqrt(np.mean(residuals ** 2))
        mse = np.mean(residuals ** 2)
        mae = np.mean(np.abs(residuals))

        # tabelize the metrics
        metrics_cleaned[location + ' \n ' + sensor_id] = [round(r2, 3), round(rmse, 3), round(mse, 3), round(mae, 3)]

    # Plot the table as an image
    fig, ax = plt.subplots(figsize=(15, 2))  # Adjust the size to fit the table
    # ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=metrics_cleaned.values, colLabels=metrics_cleaned.columns,
                     rowLabels=metrics_cleaned.index, loc='center')

    # Set font size for the entire table
    table.auto_set_font_size(False)  # Disable automatic font size
    table.set_fontsize(16)  # Set font size (adjust as needed)

    # Adjust cell width for better readability
    table.scale(1.5, 1.5)  # Scale the table (1.5x width and height)

    plt.show()

    all_metrics_cleaned[location] = metrics_cleaned

In [ ]:
# Assuming 'metrics' and 'metrics_cleaned' are already created

for metrics, metrics_cleaned in zip(all_metrics.values(), all_metrics_cleaned.values()):
    # Combine the original and cleaned metrics into a single DataFrame
    combined_metrics = pd.concat([metrics, metrics_cleaned], axis=1, keys=['Original', 'Cleaned'])
    print(combined_metrics)
    # # Plot the combined table as an image
    # fig, ax = plt.subplots(figsize=(20, 6))  # Adjust size as needed
    # ax.axis('off')  # Turn off axis
    # 
    # # Create table with multi-level column labels (Original vs Cleaned)
    # table = ax.table(cellText=combined_metrics.values, 
    #                  colLabels=combined_metrics.columns.levels[1], 
    #                  rowLabels=combined_metrics.index, 
    #                  colColours=["#ADD8E6"] * len(combined_metrics.columns.levels[1]), 
    #                  loc='center')
    # 
    # # Set font size for better readability
    # table.auto_set_font_size(False)
    # table.set_fontsize(12)
    # 
    # # Adjust the table layout
    # table.scale(1.5, 1.5)
    # 
    # plt.show()

In [ ]:
# analyse feature importance with lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Define the features and target variable
features = ['pm2p5_x', 'pressure', 'relative_humidity', 'temperature', 'wind_speed', 'rain']
target = 'pm2p5_y'

feature_importance_dict = {feature: [] for feature in features}
# Loop through each dataset and in the end plot the feature importance in a box plot
for location, df in datasets_cleaned.items():
    for sensor_id, df_sensor in df.groupby('sensor_id'):
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(df_sensor[features], df_sensor[target], test_size=0.2,
                                                            random_state=42)

        # Create a LightGBM dataset
        train_data = lgb.Dataset(X_train, label=y_train)
        test_data = lgb.Dataset(X_test, label=y_test)

        # Define the LightGBM parameters
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'seed': 42
        }

        # Train the model
        model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=1000)

        # Store the feature importance of each feature
        # Get feature importances from the model
        importances = model.feature_importance(importance_type='gain')

        # Store feature importances for this sensor
        for i, feature in enumerate(features):
            feature_importance_dict[feature].append(importances[i])

# Create a DataFrame from the feature importance dictionary
feature_importance_df = pd.DataFrame(feature_importance_dict)

In [ ]:
# Plot the feature importance as a boxplot
plt.figure(figsize=(10, 6))
feature_importance_df.boxplot(grid=False, showfliers=False)
plt.title('Feature Importance Boxplot (Gain)')
plt.ylabel('Importance')
plt.xlabel('Features')
plt.xticks(rotation=45)  # Rotate feature labels for better readability

# fix y axis
plt.ylim(0, 1.75 * 10 ** 6)

plt.show()

In [ ]:
# analyse feature importance with lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Define the features and target variable
features = ['pm2p5_x', 'pressure', 'relative_humidity', 'temperature', 'wind_speed', 'rain']
target = 'pm2p5_y'

feature_importance_dict = {feature: [] for feature in features}
# Loop through each dataset and in the end plot the feature importance in a box plot
for location, df in datasets_cleaned.items():
    for sensor_id, df_sensor in df.groupby('sensor_id'):
        # keep data where RH is under 70%
        df_sensor = df_sensor[df_sensor['relative_humidity'] < 70]

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(df_sensor[features], df_sensor[target], test_size=0.2,
                                                            random_state=42)

        # Create a LightGBM dataset
        train_data = lgb.Dataset(X_train, label=y_train)
        test_data = lgb.Dataset(X_test, label=y_test)

        # Define the LightGBM parameters
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'seed': 42
        }

        # Train the model
        model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=1000)

        # Store the feature importance of each feature
        # Get feature importances from the model
        importances = model.feature_importance(importance_type='gain')

        # Store feature importances for this sensor
        for i, feature in enumerate(features):
            feature_importance_dict[feature].append(importances[i])

# Create a DataFrame from the feature importance dictionary
feature_importance_df = pd.DataFrame(feature_importance_dict)

In [ ]:
# Plot the feature importance as a boxplot
plt.figure(figsize=(10, 6))
feature_importance_df.boxplot(grid=False, showfliers=False)
plt.title('Feature Importance Boxplot (Gain)')
plt.ylabel('Importance')
plt.xlabel('Features')
plt.xticks(rotation=45)  # Rotate feature labels for better readability

# fix y axis
plt.ylim(0, 1.75 * 10 ** 6)

plt.show()